El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Importacion de las librerias utilizadas en el proyecto

- Importamos las librerias que utilizaremos en el proyecto.

In [1]:
%%time
#Librerias iniciales
import pandas
import numpy
import matplotlib.pyplot as plt
#Librerias de Modelos y entrenamiento
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import xgboost
#Librerias de metricas
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

CPU times: user 1.43 s, sys: 353 ms, total: 1.79 s
Wall time: 1.8 s


## Preparación de datos

### Lectura de los datos

- Leemos el dataset y comprombamos sus columnas y datos

In [2]:
%%time
df=pandas.read_csv('/datasets/car_data.csv')

CPU times: user 671 ms, sys: 39.4 ms, total: 710 ms
Wall time: 719 ms


In [3]:
%%time
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
%%time
df.head(10)

CPU times: user 148 µs, sys: 26 µs, total: 174 µs
Wall time: 178 µs


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
5,04/04/2016 17:36,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,04/04/2016 00:00,0,33775,06/04/2016 19:17
6,01/04/2016 20:48,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,01/04/2016 00:00,0,67112,05/04/2016 18:18
7,21/03/2016 18:54,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,21/03/2016 00:00,0,19348,25/03/2016 16:47
8,04/04/2016 23:42,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,04/04/2016 00:00,0,94505,04/04/2016 23:42
9,17/03/2016 10:53,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,17/03/2016 00:00,0,27472,31/03/2016 17:17


- Revisamos la columna precio y esta tiene muchos valores en 0 como no nos sirve para nuestro modelo siendo el objetivo.

In [5]:
# Revisamos la columna precio
print('Tenemos',df[df['Price']==0].shape[0], 'filas de datos con un precio de valor 0')

Tenemos 10772 filas de datos con un precio de valor 0


- Revisamos la columna RegistrationYear y esta tiene años demasiados antiguos, entonces eliminaremos los datos pertenecientes a los vehiculos anteriores de 1980 ya que son solo 3542 datos, si revisamos los datos anterioes de 1990 estos son alrededor de 10000 por ende no quise eliminar desde ese año a anteriores.

In [6]:
# Revisamos la columna RegistrationYear con los datos anteriores a 1990
print('Tenemos',df[df['RegistrationYear']<1990].shape[0],'filas con año de matriculacion anteriores a 1990')

# Revisamos la columna RegistrationYear con los datos anteriores a 1980
print('Tenemos',df[df['RegistrationYear']<1980].shape[0],'filas con año de matriculacion anteriores a 1980')

Tenemos 9669 filas con año de matriculacion anteriores a 1990
Tenemos 3452 filas con año de matriculacion anteriores a 1980


### Preparacion de datos que no usaremos, duplicados y ausentes

- Revisamos cuantos datos duplicados tenemos en el dataset.

In [7]:
%%time
print('Comprobamos los datos duplicados en el dataset.')
print('El dataset tiene',df.duplicated().sum(),'duplicados.')

Comprobamos los datos duplicados en el dataset.
El dataset tiene 262 duplicados.
CPU times: user 363 ms, sys: 32.4 ms, total: 396 ms
Wall time: 407 ms


- Comprobamos cuantos datos ausentes tenemos en el dataset.

In [8]:
%%time
print('Comprobamos si hay datos nulos.')
print(df.isnull().sum())

Comprobamos si hay datos nulos.
DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64
CPU times: user 130 ms, sys: 3.51 ms, total: 133 ms
Wall time: 143 ms


- Remplazamos los datos ausentes por la cadena 'Unknown'.

In [9]:
%%time
# Completa los nombres de productos ausentes con 'Unknown'
df["VehicleType"]=df["VehicleType"].fillna("Unknown")
df["Gearbox"]=df["Gearbox"].fillna("Unknown")
#df["Model"]=df["Model"].fillna("Unknown")
df["FuelType"]=df["FuelType"].fillna("Unknown")
df["NotRepaired"]=df["NotRepaired"].fillna("Unknown")

CPU times: user 85.4 ms, sys: 3.54 ms, total: 89 ms
Wall time: 95.3 ms


- Eliminamos las caracteristicas o features (columnas) que no son necesarias para encontrar el objetivo o target (Price).

In [10]:
%%time
df=df.drop(['DateCrawled','DateCreated','LastSeen','NumberOfPictures','PostalCode','Power','Model','RegistrationMonth','Brand'],axis=1)

CPU times: user 15.9 ms, sys: 0 ns, total: 15.9 ms
Wall time: 25.8 ms


- Eliminamos los datos ausentes.

In [11]:
%%time
# Elimina los datos con precio 0
df=df.drop(df[df['Price']==0].index)
# Elimina los datos de año de registro anteriores a 1980
df=df.drop(df[df['RegistrationYear']<1980].index)
# Elimina los datos duplicados
df=df.drop_duplicates().reset_index(drop=True)
# Elimina los datos nulos
df=df.dropna().reset_index(drop=True)

CPU times: user 274 ms, sys: 8.02 ms, total: 282 ms
Wall time: 297 ms


- Comprobamos que todas las operaciones se realizaron correctamente.

In [12]:
%%time
print('Comprobamos los datos duplicados en el dataset.')
print('El dataset tiene',df.duplicated().sum(),'duplicados.')
print("")
print('Comprobamos si hay datos nulos.')
print(df.isnull().sum())
df.info()

Comprobamos los datos duplicados en el dataset.
El dataset tiene 0 duplicados.

Comprobamos si hay datos nulos.
Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Mileage             0
FuelType            0
NotRepaired         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184320 entries, 0 to 184319
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             184320 non-null  int64 
 1   VehicleType       184320 non-null  object
 2   RegistrationYear  184320 non-null  int64 
 3   Gearbox           184320 non-null  object
 4   Mileage           184320 non-null  int64 
 5   FuelType          184320 non-null  object
 6   NotRepaired       184320 non-null  object
dtypes: int64(3), object(4)
memory usage: 9.8+ MB
CPU times: user 157 ms, sys: 4.02 ms, total: 161 ms
Wall time: 171 ms


### Codificacion de los datos

- Extraemos un lista con las columnas categoricas del dataframe

In [13]:
%%time
# Extraemos columnas categoricas del DataFrame
categorical_columns=df.select_dtypes(include=['object']).columns.tolist()
print(categorical_columns)

['VehicleType', 'Gearbox', 'FuelType', 'NotRepaired']
CPU times: user 11.5 ms, sys: 466 µs, total: 12 ms
Wall time: 10.3 ms


- Realizamos el encodamiento de los datos que son cadenas (Strings) con OrdinalEncoder

In [14]:
%%time
# Realizamos la codificacion a nuestro dataFrame
ordinal_encoder = OrdinalEncoder()
# Creamos un DataFrame con las columnas OrdinalEncoder
df_ordinal=pandas.DataFrame(ordinal_encoder.fit_transform(df),columns=df.columns)

CPU times: user 321 ms, sys: 3.8 ms, total: 325 ms
Wall time: 344 ms


- Realizamos el encodamiento de los datos que son cadenas (Strings) con OneHotEncoder

In [15]:
%%time
# Realizamos la codificacion a nuestro dataFrame
one_hot_encoder =OneHotEncoder()
# Aplicamos OHE en las columnas categoricas
ohe=one_hot_encoder.fit_transform(df[categorical_columns])
# Creamos un DataFrame con las columnas OHE
ohe_data=pandas.DataFrame(ohe.toarray(),columns=one_hot_encoder.get_feature_names(categorical_columns))
# Concatenamos ohe_df con el df original
df_ohe=pandas.concat([df,ohe_data],axis=1)
# Eliminamos las columnas categoricas originales
df_ohe=df_ohe.drop(categorical_columns,axis=1)

CPU times: user 290 ms, sys: 24 ms, total: 314 ms
Wall time: 324 ms


###  Estandarizado de datos

- Creamos una variable de la clase StandardScaler

In [16]:
%%time
# Estandarizamos los datos con el StandardScaler
scaler = StandardScaler()

CPU times: user 11 µs, sys: 1 µs, total: 12 µs
Wall time: 16 µs


- Realizamos el estandarizado de la data que fue hecha encoding con OrdinalEncoder

In [17]:
%%time
# Realizamos el metodo solo en las columnas no binarias resultantes del encoder(osea todas las columnas)
df_ordinal=pandas.DataFrame(scaler.fit_transform(df_ordinal), columns=df_ordinal.columns)
# Verificamos el nuevo DataFrame
df_ordinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184320 entries, 0 to 184319
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             184320 non-null  float64
 1   VehicleType       184320 non-null  float64
 2   RegistrationYear  184320 non-null  float64
 3   Gearbox           184320 non-null  float64
 4   Mileage           184320 non-null  float64
 5   FuelType          184320 non-null  float64
 6   NotRepaired       184320 non-null  float64
dtypes: float64(7)
memory usage: 9.8 MB
CPU times: user 71.4 ms, sys: 347 µs, total: 71.8 ms
Wall time: 93 ms


- Realizamos el estandarizado de la data que fue hecha encoding con OneHotEncoder

In [18]:
%%time
# Realizamos el metodo solo en las columnas no binarias resultantes del encoder(osea todas las columnas)
df_ohe=pandas.DataFrame(scaler.fit_transform(df_ohe), columns=df_ohe.columns)
# Verificamos el nuevo DataFrame
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184320 entries, 0 to 184319
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Price                    184320 non-null  float64
 1   RegistrationYear         184320 non-null  float64
 2   Mileage                  184320 non-null  float64
 3   VehicleType_Unknown      184320 non-null  float64
 4   VehicleType_bus          184320 non-null  float64
 5   VehicleType_convertible  184320 non-null  float64
 6   VehicleType_coupe        184320 non-null  float64
 7   VehicleType_other        184320 non-null  float64
 8   VehicleType_sedan        184320 non-null  float64
 9   VehicleType_small        184320 non-null  float64
 10  VehicleType_suv          184320 non-null  float64
 11  VehicleType_wagon        184320 non-null  float64
 12  Gearbox_Unknown          184320 non-null  float64
 13  Gearbox_auto             184320 non-null  float64
 14  Gear

## Entrenamiento de los modelos

- Creamos los conjuntos de entrenamiento, validacion y pruebas con la proporcion 3:1:1 (60,20,20) para los datos hechos con OrdinalEncoder

In [19]:
%%time
# Manera investigada para dividir en 3 conjuntos con proporcion 3:1:1.
# Obtenemos el conjunto de 'test' y dejamos otro conjunto llamado 'rest'.
rest_ordinal,test_ordinal=train_test_split(df_ordinal,test_size=0.2,random_state=54321)
# Del conjunto 'rest' obtendemos el conjunto de entrenamiento y validacion.
train_ordinal,valid_ordinal=train_test_split(rest_ordinal,test_size=0.25,random_state=54321)

CPU times: user 38.8 ms, sys: 3.83 ms, total: 42.6 ms
Wall time: 53 ms


- Creamos los conjuntos de entrenamiento, validacion y pruebas con la proporcion 3:1:1 (60,20,20) para los datos hechos con OrdinalEncoder

In [20]:
%%time
# Manera investigada para dividir en 3 conjuntos con proporcion 3:1:1.
# Obtenemos el conjunto de 'test' y dejamos otro conjunto llamado 'rest'.
rest_ohe,test_ohe=train_test_split(df_ohe,test_size=0.2,random_state=54321)
# Del conjunto 'rest' obtendemos el conjunto de entrenamiento y validacion.
train_ohe,valid_ohe=train_test_split(rest_ohe,test_size=0.25,random_state=54321)

CPU times: user 82.4 ms, sys: 8.03 ms, total: 90.4 ms
Wall time: 97.8 ms


- En los conjuntos separamos las caracteristicas (features) y el objetivo (target) para los conjuntos de OrdinalEncoder.

In [21]:
%%time
#Entrenamiento
features_train_ordinal=train_ordinal.drop(['Price'],axis=1)
target_train_ordinal=train_ordinal['Price']
#Validacion
features_valid_ordinal=valid_ordinal.drop(['Price'],axis=1)
target_valid_ordinal= valid_ordinal['Price']
#Prueba
features_test_ordinal=test_ordinal.drop(['Price'],axis=1)
target_test_ordinal=test_ordinal['Price']

CPU times: user 2.7 ms, sys: 4.06 ms, total: 6.75 ms
Wall time: 6.57 ms


- En los conjuntos separamos las caracteristicas (features) y el objetivo (target) para los conjuntos de OneHotEncoder.

In [22]:
%%time
#Entrenamiento
features_train_ohe=train_ohe.drop(['Price'],axis=1)
target_train_ohe=train_ohe['Price']
#Validacion
features_valid_ohe=valid_ohe.drop(['Price'],axis=1)
target_valid_ohe= valid_ohe['Price']
#Prueba
features_test_ohe=test_ohe.drop(['Price'],axis=1)
target_test_ohe=test_ohe['Price']

CPU times: user 7.42 ms, sys: 8.31 ms, total: 15.7 ms
Wall time: 14.5 ms


### Modelo: Arbol de decisión

- Nuestro primer modelo a entrenar sera el arbol de decisión con los conjuntos de OrdinalEncoder..

In [23]:
%%time
# Declaramos variables para medir mce y exactitud de los modelos.
best_score_tree=0
best_depth_tree=0
best_mse_tree=0
for depth_tree in range(1,6): # Seleccionamos el rango del hiperparámetro.
    model_tree= DecisionTreeRegressor(max_depth=depth_tree,random_state=12345) # Declaramos el modelo.
    model_tree.fit(features_train_ordinal,target_train_ordinal) # Entrenamos el modelo en el conjunto de entrenamiento.
    predictions_valid_tree = model_tree.predict(features_valid_ordinal) # Obtenemos las predicciones del modelo en el conjunto de validación.
    mse_tree =mean_squared_error(target_valid_ordinal,predictions_valid_tree)**0.5# calcula la RECM en el conjunto de validación
    score_tree= model_tree.score(features_test_ordinal,target_test_ordinal) # Calculamos la puntuacion de accuracy en el conjunto de test.
    if mse_tree > best_mse_tree:
        best_mse_tree = mse_tree
        best_depth__tree = depth_tree
    if score_tree>best_score_tree:
        best_score_tree=score_tree # Guardamos la mejor puntuacion de accurracy en el conjunto de validación.

print("El RECM del mejor modelo en el conjunto de validación:", best_mse_tree,"y mejor profundidad:", best_depth_tree)

print("La exactitud del mejor modelo en el conjunto de pruebas es de : {}".format(best_score_tree))

El RECM del mejor modelo en el conjunto de validación: 0.8213823509796854 y mejor profundidad: 0
La exactitud del mejor modelo en el conjunto de pruebas es de : 0.5519236868107
CPU times: user 368 ms, sys: 4.06 ms, total: 372 ms
Wall time: 398 ms


### Modelo: Bosque aleatorio

- Nuestro segundo modelo a entrenar sera el bosque aleatorio con los conjuntos de OrdinalEncoder..

In [24]:
%%time
# Declaramos variables para medir mce y exactitud de los modelos.
best_score_forest=0
best_est_score_forest=0
best_est_mse_forest=0
best_depth_forest=0
best_mse_forest=0
for est_forest in range(5,20,5): # Entrenaremos 20 modelos en intervalos de 5
    for depth_forest in range (1,5): # La profundidad de los arboles sera de 1 a 5
        model_forest=RandomForestRegressor(random_state=54321,n_estimators=est_forest, max_depth=depth_forest) # Configuramos el numero de arboles.
        model_forest.fit(features_train_ordinal,target_train_ordinal) # Entrenamos el modelo con el conjunto de entrenamiento.
        predictions_valid_forest= model_forest.predict(features_valid_ordinal) # Realizamos prediciones con el conjunto de validación.
        mse_forest=mean_squared_error(target_valid_ordinal,predictions_valid_forest)**0.5 # Calculamos el ECM
        score_forest= model_forest.score(features_test_ordinal,target_test_ordinal) # Calculamos la puntuacion de accuracy en el conjunto de test.
        if score_forest>best_score_forest:
            best_score_forest=score_forest # Guardamos la mejor puntuacion de accurracy en el conjunto de validación.
            best_est_score_forest=est_forest # Guardamos el número de estimadores que corresponden a la mejor punturación de exactitud.
        if mse_forest>best_mse_forest:
            best_mse_forest=mse_forest # Guardamos la mejor prueba de MCE.
            best_est_mse_forest=est_forest # Guardamos el número de estimadores que corresponden a la mejor prueba de MCE.
            best_depth_forest= depth_forest # Guardamos la mejor profundidad para los arboles del bosque.

print("El RECM del mejor modelo en el conjunto de validación:", best_mse_forest, "con estimadores de:", best_est_mse_forest, "y mejor profundidad:", best_depth_forest)

print("La exactitud del mejor modelo en el conjunto de pruebas con n_estimators(estimadores) {}) es de : {}".format(best_est_score_forest, best_score_forest))

El RECM del mejor modelo en el conjunto de validación: 0.8213765420349456 con estimadores de: 5 y mejor profundidad: 1
La exactitud del mejor modelo en el conjunto de pruebas con n_estimators(estimadores) 10) es de : 0.5243334473212868
CPU times: user 4.78 s, sys: 0 ns, total: 4.78 s
Wall time: 4.8 s


### Modelo: Regresion lineal

- Nuestro ultimo modelo a entrenar sera el modelo de regresion lineal los conjuntos de OneHotEncoder..

In [25]:
%%time
model_lr= LinearRegression()
model_lr.fit(features_train_ohe,target_train_ohe)
predictions_valid_lr=model_lr.predict(features_valid_ohe)# Obtenemos las predicciones del modelo en el conjunto de validación.
rmse_lr=mean_squared_error(target_valid_ohe,predictions_valid_lr)**0.5
print('El RECM del modelo es:',rmse_lr)

El RECM del modelo es: 0.8112262428249876
CPU times: user 136 ms, sys: 37.8 ms, total: 173 ms
Wall time: 134 ms


### Modelo: Regresion LGBM

- El siguiente modelo a entrenar sera el modelo LGBMRegressor con los conjuntos de OrdinalEncoder.

In [26]:
%%time
# Creamos una instancia del regresor LightGBM con la métrica RMSE.
model_lgbm = LGBMRegressor(metric='rmse') 

# Entrenamos el modelo utilizando los datos de entrenamiento.
model_lgbm.fit(features_train_ordinal, target_train_ordinal) 
  
# Hacer predicciones sobre los datos de validacion y test.
predict_lgbm_valid = model_lgbm.predict(features_valid_ordinal) 
predict_lgbm_test = model_lgbm.predict(features_test_ordinal) 
# Obtenemos el RMSE
rmse_lgbm_valid = mean_squared_error(target_valid_ordinal,predict_lgbm_valid)**0.5
rmse_lgbm_test = mean_squared_error(target_test_ordinal,predict_lgbm_test)**0.5 
# Mostramos los RMSE
print('RECM del conjunto de validacion:',rmse_lgbm_valid)
print('RECM del conjunto de validacion:',rmse_lgbm_test)

RECM del conjunto de validacion: 0.569797270664516
RECM del conjunto de validacion: 0.5714442709591238
CPU times: user 1min 12s, sys: 692 ms, total: 1min 13s
Wall time: 1min 13s


### Modelo: Regresion CatBoost

- El siguiente modelo a entrenar sera el modelo CatBoostRegressor con los conjuntos de OrdinalEncoder.

In [27]:
%%time
# Creamos una instancia del CatBoostRegressor con RMSE como la loss function 
model_catboost = CatBoostRegressor(loss_function='RMSE') 
  
# Ajustar el modelo a los datos de entrenamiento con un registro detallado cada 100 iteraciones 
model_catboost.fit(features_train_ordinal, target_train_ordinal, verbose=100)

# Hacer predicciones sobre los datos de validacion y test.
predict_cat_valid = model_catboost.predict(features_valid_ordinal) 
predict_cat_test = model_catboost.predict(features_test_ordinal) 
# Obtenemos el RMSE
rmse_cat_valid = mean_squared_error(target_valid_ordinal,predict_cat_valid)**0.5
rmse_cat_test = mean_squared_error(target_test_ordinal,predict_cat_test)**0.5 
# Mostramos los RMSE
print('RECM del conjunto de validacion:',rmse_cat_valid)
print('RECM del conjunto de validacion:',rmse_cat_test)

Learning rate set to 0.086117
0:	learn: 0.9562291	total: 68.2ms	remaining: 1m 8s
100:	learn: 0.5749550	total: 1.98s	remaining: 17.6s
200:	learn: 0.5674267	total: 3.89s	remaining: 15.5s
300:	learn: 0.5643445	total: 5.91s	remaining: 13.7s
400:	learn: 0.5620539	total: 7.96s	remaining: 11.9s
500:	learn: 0.5604993	total: 9.96s	remaining: 9.92s
600:	learn: 0.5592314	total: 12s	remaining: 7.99s
700:	learn: 0.5581068	total: 13.8s	remaining: 5.88s
800:	learn: 0.5570836	total: 15.5s	remaining: 3.86s
900:	learn: 0.5562830	total: 17.2s	remaining: 1.89s
999:	learn: 0.5555291	total: 18.9s	remaining: 0us
RECM del conjunto de validacion: 0.5678028654364454
RECM del conjunto de validacion: 0.5691080517922429
CPU times: user 18.9 s, sys: 79.2 ms, total: 18.9 s
Wall time: 19.3 s


### Modelo:  XGBoost

- El ultimo modelo a entrenar sera el modelo XGBoost con su propio encoding que es transformando sus columnas 'object' en 'category'.

- Comencamos copiando el DataFrame Original en uno nuevo para realizar los conjuntos de entrenamiento.

In [28]:
%%time
df_xgboost=df
df_xgboost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184320 entries, 0 to 184319
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             184320 non-null  int64 
 1   VehicleType       184320 non-null  object
 2   RegistrationYear  184320 non-null  int64 
 3   Gearbox           184320 non-null  object
 4   Mileage           184320 non-null  int64 
 5   FuelType          184320 non-null  object
 6   NotRepaired       184320 non-null  object
dtypes: int64(3), object(4)
memory usage: 9.8+ MB
CPU times: user 41.2 ms, sys: 0 ns, total: 41.2 ms
Wall time: 38.9 ms


- Transformamos las columnas 'object' en 'category'

In [29]:
%%time
df_xgboost['VehicleType'] = df_xgboost['VehicleType'].astype('category')
df_xgboost['Gearbox'] = df_xgboost['Gearbox'].astype('category')
df_xgboost['FuelType'] = df_xgboost['FuelType'].astype('category')
df_xgboost['NotRepaired'] = df_xgboost['NotRepaired'].astype('category')

CPU times: user 69.9 ms, sys: 0 ns, total: 69.9 ms
Wall time: 91.8 ms


- Realziamos los conjuntos de entrenamiento para el modelo.

In [30]:
%%time
# Manera investigada para dividir en 3 conjuntos con proporcion 3:1:1.
# Obtenemos el conjunto de 'test' y dejamos otro conjunto llamado 'rest'.
rest_xgboost,test_xgboost=train_test_split(df_xgboost,test_size=0.2,random_state=54321)
# Del conjunto 'rest' obtendemos el conjunto de entrenamiento y validacion.
train_xgboost,valid_xgboost=train_test_split(rest_xgboost,test_size=0.25,random_state=54321)

CPU times: user 35.7 ms, sys: 39 µs, total: 35.7 ms
Wall time: 46 ms


- Separamos caracteristicas y objetivos de los conjuntos obtenidos.

In [31]:
%%time
#Entrenamiento
features_train_xgboost=train_xgboost.drop(['Price'],axis=1)
target_train_xgboost=train_xgboost['Price']
#Validacion
features_valid_xgboost=valid_xgboost.drop(['Price'],axis=1)
target_valid_xgboost= valid_xgboost['Price']
#Prueba
features_test_xgboost=test_xgboost.drop(['Price'],axis=1)
target_test_xgboost=test_xgboost['Price']

CPU times: user 5.2 ms, sys: 16 µs, total: 5.21 ms
Wall time: 4.02 ms


- Comenzamos con el entrenamiento del modelo donde hay que hacer cierta modificacion en los conjuntos para que puedan entrenar al modelo.

In [32]:
%%time
# Convertimos el dataset en una DMatrix(Segun la documentacion), recordemos que ya no tenemos columnas categoricas.

xgb_train=xgboost.DMatrix(features_train_xgboost,target_train_xgboost,enable_categorical=True)
xgb_valid=xgboost.DMatrix(features_valid_xgboost,target_valid_xgboost,enable_categorical=True)
xgb_test=xgboost.DMatrix(features_test_xgboost,target_test_xgboost,enable_categorical=True)

# Diccionario de parametros que utilizaremos en el modelo
n=50
params={'objective': 'reg:squarederror','max_depth': 3,'learning_rate': 0.1,}

# Creamos una instancia del XGBoost
model_xgboost = xgboost.train(params=params,dtrain=xgb_train,num_boost_round=n) 

# Hacer predicciones sobre los datos de validacion y test.
predict_xgb_valid = model_xgboost.predict(xgb_valid)
predict_xgb_test = model_xgboost.predict(xgb_test)

# Transformamos en entero para un calculo mas sencillo
predict_xgb_valid = predict_xgb_valid.astype(int)
predict_xgb_test = predict_xgb_test.astype(int)

# Obtenemos la exactitud del modelo
rmse_xgboost_valid = mean_squared_error(target_valid_xgboost,predict_xgb_valid)*0.5
rmse_xgboost_test = mean_squared_error(target_test_xgboost,predict_xgb_test)*0.5
# Mostramos los RMSE
print('RECM del modelo con el conjunto de validacion es:', rmse_xgboost_valid)
print('RECM del modelo con el conjunto de prueba es:', rmse_xgboost_test)

RECM del modelo con el conjunto de validacion es: 4752078.646443685
RECM del modelo con el conjunto de prueba es: 4878473.679633247
CPU times: user 6.72 s, sys: 50.7 ms, total: 6.77 s
Wall time: 6.78 s


## Análisis del modelo

- Podemos observar un resumen de todos los RECM de los modelos entrenados para su comparacion sencilla de un solo vistazo.

In [33]:
%%time
print("El RECM del modelo Árbol de decision en el conjunto de validación:", best_mse_tree,"y mejor profundidad:", best_depth_tree)
print("La exactitud del modelo  Árbol de decision en el conjunto de pruebas es de : {}".format(best_score_tree))
print()
print("El RECM del modelo bosque aleatorio en el conjunto de validación:", best_mse_forest, "con estimadores de:", best_est_mse_forest, "y mejor profundidad:", best_depth_forest)
print("La exactitud del modelo bosque aleatorio en el conjunto de pruebas con n_estimators(estimadores) {}) es de : {}".format(best_est_score_forest, best_score_forest))
print()
print('El RECM del modelo es:',rmse_lr)
print()
print('RECM del conjunto de validacion:',rmse_lgbm_valid)
print('RECM del conjunto de validacion:',rmse_lgbm_test)
print()
print('RECM del conjunto de validacion:',rmse_cat_valid)
print('RECM del conjunto de validacion:',rmse_cat_test)
print()
print('RECM del modelo con el conjunto de validacion es:', rmse_xgboost_valid)
print('RECM del modelo con el conjunto de prueba es:', rmse_xgboost_test)

El RECM del modelo Árbol de decision en el conjunto de validación: 0.8213823509796854 y mejor profundidad: 0
La exactitud del modelo  Árbol de decision en el conjunto de pruebas es de : 0.5519236868107

El RECM del modelo bosque aleatorio en el conjunto de validación: 0.8213765420349456 con estimadores de: 5 y mejor profundidad: 1
La exactitud del modelo bosque aleatorio en el conjunto de pruebas con n_estimators(estimadores) 10) es de : 0.5243334473212868

El RECM del modelo es: 0.8112262428249876

RECM del conjunto de validacion: 0.569797270664516
RECM del conjunto de validacion: 0.5714442709591238

RECM del conjunto de validacion: 0.5678028654364454
RECM del conjunto de validacion: 0.5691080517922429

RECM del modelo con el conjunto de validacion es: 4752078.646443685
RECM del modelo con el conjunto de prueba es: 4878473.679633247
CPU times: user 440 µs, sys: 6 µs, total: 446 µs
Wall time: 418 µs
